# PA1
#### Applied Machine Learning
Grpup 39: Sebastian Kölbel & Min Ze Teh
## Task1
We begin by using the code provided in the assignment to transform the data in the csv file into a panda dataframe.


In [54]:
import pandas as pd
from sklearn.model_selection import train_test_split
  
# Read the CSV file.
data = pd.read_csv('CTG.csv', skiprows=1)

# Select the relevant numerical columns.
selected_cols = ['LB', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'ASTV', 'MSTV', 'ALTV',
                 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean',
                 'Median', 'Variance', 'Tendency', 'NSP']
data = data[selected_cols].dropna()

# Shuffle the dataset.
data_shuffled = data.sample(frac=1.0, random_state=0)

# Split into input part X and output part Y.
X = data_shuffled.drop('NSP', axis=1)

# Map the diagnosis code to a human-readable label.
def to_label(y):
    return [None, 'normal', 'suspect', 'pathologic'][(int(y))]

Y = data_shuffled['NSP'].apply(to_label)

# Partition the data into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)

When choosing three classifiers we decided to pick one classifier form each of the different sub categories. We picked gradient boosting classifier and attempted to optimize it in terms of max depth, trying out different depths we found that we got the best performance when having a max depth of 5 or 6.

For the Multi-layer perceptron classfier we tried adjusting the hidden_layer_sizes and found that we got the best accuracy when using the input value 75.

In [55]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.linear_model import Perceptron as perc
from sklearn.neural_network import MLPClassifier as mlpc

gbc_clf = gbc(max_depth=5,random_state=1)
perc_clf = perc() 
mlpc_clf = mlpc(random_state= 1, hidden_layer_sizes=75)

print('Gradient boosting classifier accuracy :', cross_val_score(gbc_clf, Xtrain, Ytrain).mean())
print('Perceptron accuracy:', cross_val_score(perc_clf, Xtrain, Ytrain).mean())
print('MLP Classifier accuracy:', cross_val_score(mlpc_clf, Xtrain, Ytrain).mean())

Gradient boosting classifier accuracy : 0.95
Perceptron accuracy: 0.825294117647059
MLP Classifier accuracy: 0.8841176470588236


For the final evaluation we picked the classifier that we found to have the best accuracy, namely the gradient boosting classifier. we fit the values and performed the prediction the resulting accuracy can be seen below.

In [56]:
from sklearn.metrics import accuracy_score

gbc_clf.fit(Xtrain, Ytrain)
Yguess = gbc_clf.predict(Xtest)
print('Gradient boosting classifier accuracy: ',accuracy_score(Ytest, Yguess))

Gradient boosting classifier accuracy:  0.931924882629108


## Task 2


In [ ]:
class DecisionTreeLeaf:

    def __init__(self, value):
        self.value = value

    # This method computes the prediction for this leaf node. This will just return a constant value.
    def predict(self, x):
        return self.value

    # Utility function to draw a tree visually using graphviz.
    def draw_tree(self, graph, node_counter, names):
        node_id = str(node_counter)
        val_str = f'{self.value:.4g}' if isinstance(self.value, float) else str(self.value)
        graph.node(node_id, val_str, style='filled')
        return node_counter+1, node_id
        
    def __eq__(self, other):
        if isinstance(other, DecisionTreeLeaf):
            return self.value == other.value
        else:
            return False